In [0]:
# import pandas and language detection module
import langid
import pandas as pd

# read entire csv file, read ids of tweets identified to be in English
df = pd.read_csv('ghana_election.csv')
english_df = pd.read_csv('idx_english_tweets.csv')

In [0]:
# ensure report row is of type string
df['report'] = df['report'].astype(str)

In [0]:
# get all English tweets
df_english_tweets = df[df.index.isin(english_df['english_id'])]

In [0]:
# function to remove retweets

def remove_retweets():
  
  # set of tweets with retweets, or in some cases, they are retweets with the original missing
  original_tweets = set()   

  k = len(df_english_tweets)
  i = 0

  # to store index of all tweets
  all_tweets_idx = []

  # for each row in English tweets dataframe
  for index, row  in df_english_tweets.iterrows():    

    # get the actual tweet 
    tweet = row['report']

    # keep count to see what's going on
    if (i % 1000 == 1):
      print("Processing tweet {} of {}".format(i, k))
    i = i + 1

    # we will add new tweets to original tweet if they meet some criteria
    for original_tweet in original_tweets.copy(): 
      
      # if current tweet is a superset of some original tweet and it starts with RT, quit loop and go to the next tweet, its most likely a retweet
      if original_tweet in tweet and tweet.startswith('RT'):  
        break
      
      # if original tweet is a superset of the current tweet, then remove original tweet
      elif tweet in original_tweet and original_tweet.startswith('RT'): 
        original_tweets.remove(original_tweet)
    
    # add current tweet to list of original tweet
    else:                                                         
      original_tweets.add(tweet)
      all_tweets_idx.append(index)
  
  # combine original tweets with already labelled tweets
  lab_unl_tweets_idx = set(df_labelled_tweets.index).union(all_tweets_idx)

  # find the corresponding rows in the main df
  final_tweets = [df.loc[tweet_idx].tolist() for tweet_idx in lab_unl_tweets_idx]
  return final_tweets


In [0]:
# all original tweets + labelled tweets
res = remove_retweets()

In [0]:
# save to tweets.csv
res_tweets = pd.DataFrame(res, columns=['report', 'source', 'authored_at', 'author','status','incident_title','incident_category','important','place_name',	'note'])
res_tweets.to_csv('tweets.csv', index=False)